<a href="https://colab.research.google.com/github/yecatstevir/teambrainiac/blob/main/source/SingleSubjectSVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Whole Brain Support Vector Machine Training
- Go to 'Runtime' in Colab browser bar, select 'Change Runtime Type', select 'High-RAM' from 'Runtime Shape'. 
- load local pickle file containing all masked, normalized Whole Brain subject data in numpy matrix format
- SVM training per subject

### Mount Google Drive and clone repository
- open to source directory

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')#, force_remount = True)

Mounted at /content/gdrive


In [2]:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
# Clone the entire repo.
!git clone -l -s https://github.com/yecatstevir/teambrainiac.git
# Change directory into cloned repo
%cd teambrainiac/source
!ls


Cloning into 'teambrainiac'...
remote: Enumerating objects: 1387, done.
remote: Counting objects: 100% (1387/1387), done.
remote: Compressing objects: 100% (1089/1089), done.
remote: Total 1387 (delta 895), reused 552 (delta 281), pack-reused 0
Receiving objects: 100% (1387/1387), 84.69 MiB | 11.74 MiB/s, done.
Resolving deltas: 100% (895/895), done.
/content/teambrainiac/source
access_data.py			  models
AccuracyMeasures.ipynb		  process.py
data				  single_subject.py
DataExploration_SingleSubj.ipynb  SingleSubjectSVM.ipynb
DataExplorationVisuals.ipynb	  streamlit
DL				  SubjectVisualization_Models_ZNORM.ipynb
group_svm			  TestMask.ipynb
helper				  utils.py
images				  VisualizationPlayground.ipynb
__init__.py


### Load path_config.py 
- we are already in source so we can just load this file without changing directory

In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving path_config.py to path_config.py
User uploaded file "path_config.py" with length 228 bytes


### Import libraries


In [6]:


# Import libraries
!pip install boto3 nilearn nibabel #for saving data and image visualizations
import pickle
#sklearn packages needed
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, auc, recall_score, precision_score,roc_curve,f1_score
#important utility functions for loading,masking,saving data
#from utils import *
from access_data import *
from process import *
#normal python packages we use
import numpy as np
import pandas as pd
from scipy import signal
from nilearn.signal import clean

     |████████████████████████████████| 132 kB 4.3 MB/s 
     |████████████████████████████████| 9.6 MB 17.6 MB/s 
     |████████████████████████████████| 79 kB 7.9 MB/s 
     |████████████████████████████████| 8.7 MB 54.7 MB/s 
     |████████████████████████████████| 138 kB 74.2 MB/s 
     |████████████████████████████████| 38.1 MB 5.3 MB/s 
     |████████████████████████████████| 127 kB 65.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 r

### Get paths to subject data and grab labels for SVM

In [7]:

## load and open the pickle file that contains paths to all data.
path = "data/data_path_dictionary.pkl"
data_path_dict = open_pickle(path)

##get mask_dictionary


###Functions to get information about data to run our SVM

In [8]:
def get_data_dict(path):
  """
    Function to get data path dict
      params:
        path : str: path to data path dictionary
      returns: dictionary of data paths
  """
  data_path_dict = open_pickle(path)
  return data_path_dict

def get_subj_information(data_path_dict):
  """
    Function to get subject information.
    data_path_dict  : dictionary containing paths to all data stored on AWS
    returns:  subject_ids(list of subjects to run),subj_paths(paths to subject raw data)
  """
  subject_ids = data_path_dict['subject_ID'] #subject_ids
  subj_paths = data_path_dict['subject_data'] #subject_paths
  return subject_ids,subj_paths

def get_labels(data_path_dict):
  """
    Function to get the labels for our data.
    data_path_dict  : dictionary containing paths to all data stored on AWS
    returns: mask_labels_indices(timepoints we want masked out),binary_labels(labels for our for our two brain states)
             and label_type
  """
  
  label_data_path = data_path_dict['labels'][0] #get labels
  label_type = 'rt_labels' #tell the function what labels we want
  mask_labels_indices, binary_labels = labels_mask_binary(label_data_path, label_type) #grab indices and labels
  return mask_labels_indices, binary_labels,label_type

def get_mask_data(data_path_dict,mask_ind):
  """
    Function to return the mask of what brain voxels we want to include in analysis
    Params:
      data_path_dict  : dictionary: containing paths to data
      mask_ind: int: index of where the path to the masks are 0: full brain mask plus masks that subtract region
                1: Regions of interest(ROIs) mask out full brain except structure we care about
    returns: dictionary: contains mask data
    
  """
  mask_data_filepath = data_path_dict['mask_data'][mask_ind] #path to masked data     
  mask_type_dict = access_load_data(mask_data_filepath, True) #get the mask data dictionary
  
  return mask_type_dict

In [9]:
def make_mask(np_array_mask):
  """
    Function to create boolean mask to mask out voxels we don't want
    Params:
      mask_type: string: which mask to grab to get boolean array
    returns: boolean array of voxels to include
  """
  #np_array_mask = mask_data[mask_type] #get the mask array
  #create a 1-D array for the mask. Important to use Fourier Transformation as we are working in brain space!
  mask = np.ma.make_mask(np_array_mask).reshape(79*95*79,order='F')
  return mask

## Set up SVM Model

In [10]:
def mask_subject_data(data,mask,mask_labels_indices):
  """
    Function to mask user data to mask out voxels we don't want
    Params:
      data: dictionary: subject data dictionary contain 4 runs of unmasked data
      mask: nd.array: 1-d array boolean values used to only include voxels we want.
      mask_labels_indices: indices of rows we want in to include in our model
    returns: dictionary: includes 4 runs of masked data
  """
  user_data_dict = {} #create empty dict
  arr = []
  for i in tqdm.tqdm(range(4)):
      user_key = 'run_0' + str(i+1) + '_vec'
      array = data[user_key]
      array_masked = array[:, mask]
      array_masked = array_masked[mask_labels_indices]  
      arr.append(array_masked)
  user_data_dict['data'] = arr
  return user_data_dict

In [11]:
def scale_data_single_subj(sub_data,train_runs,test_runs,norm='none'):
  """
    Function to scale data.
    Params:
      sub_data     : (1 subject data, keys as subject ID for frmi data or labels)
      sub_id       : subject id  of subject we are normalizing for
      runs_test    : tuple, (which run are we using for the test data)
      norm         : list, ("RUNS": normalizing separately on each run;
                              "SUBJECT": Normalizing separately by each subject)
    returns      : dictionary of nd.arrays, Concatenated X data of (time points, x*y*z) x = 79, y = 95, z = 75
                    and Concatenated y labels of (time points,)
    """
  ##run standardization
  ##initialize empty dictionary
  normalized_runs = {}
  for run in runs_list:
    run_name = user_key = 'run_0' + str(run) 
    run_data = sub_data['data'][run-1]
    if norm=='none':
      normalized_runs[run_name] = clean(run_data,detrend=True,standardize=False,filter=False,standardize_confounds=False)
    else:
      normalized_runs[run_name] = clean(run_data,detrend=True,standardize=norm,filter=False,standardize_confounds=False)
  return normalized_runs

In [12]:
def get_accuracy_scores(clf,data,X_train,y_train,runs_test,y_labels):
  """
    Function to get accuracy scores for subject models.
    Params:
      model_dict: contains subject model and training/test/val/data
      subj: subject name 
      normalization_type: options: 'PSC','ZNORM','none' what type of normalization
    returns: subj_list, list of subject metrics
  """
  accuracy_list = []
  df_columns = ['train_acc']
  y_predicts = clf.predict(X_train)
  accuracy_list.append(accuracy_score(y_train,y_predicts))
  for run in runs_test:
    y_predicts = clf.predict(data[run])
    df_columns.append(run + '_acc')
    accuracy_list.append(accuracy_score(y_labels,y_predicts))
    df_columns.append(run+'_f1_score')
    accuracy_list.append(f1_score(y_labels,y_predicts))
    
    
  return accuracy_list,df_columns

In [20]:
def get_predicts(clf,data,runs_test):
  """
    Function to get accuracy scores for subject models.
    Params:
      model_dict: contains subject model and training/testdata
      subj: subject name 
    returns: y_val_predicts(if validation run),y_test_predicts
  """
  predictions_dict = {}
  for runs in runs_test:
    predictions_dict[runs] = {}
    predictions_dict[runs]['predicts'] = clf.predict(data[runs])
    predictions_dict[runs]['proba'] = clf.predict_proba(data[runs])
    predictions_dict[runs]['decision_function'] = clf.decision_function(data[runs])
  
                                                  
  return predictions_dict

In [14]:
def run_single_subject_svm(data,runs_train,train_labels,svc_kernel='rbf',svc_c=1,do_cv=False,params={}):
  """
    Function to run cross-validation or single subject SVM
    Params:
      tuple: contains
        X_train      : 2-d array of training data
        y_train   : sub_labels to indicate which row of the sub_data belongs to increase/decrease state
        svc_kernel : kernel for svc
        svc_c: c value for svc
      optionals:
        do_cv: boolean: to decide if cross-validation gridsearch is requested: default=False
        params: dictionary: dictionary containing params to grid search: default=empty dictionary
    returns      : subject individual model
  """ 
  #run cv if do_cv = True, else run individual model SVM
  X_train = []
  y_train = []
  if len(runs_train)>1:
        for run in runs_train:
          X_train.append(data[run])
          y_train.append(train_labels)    
        X_train = np.concatenate(np.array(X_train))
        y_train = np.concatenate(np.array(y_train))
  else:
    X_train = data[runs_train[0]]
    y_train = train_labels
  if do_cv:
    #cv_params = {'C':[0.7, 1, 5, 10],'kernel':['linear', 'rbf']}
    svc = SVC()
    clf = GridSearchCV(svc, params)
    clf.fit(X_train,y_train)
    return clf
  else:
    clf = SVC(C=svc_c,kernel=svc_kernel,probability=True)
    clf.fit(X_train,y_train)
  return clf,X_train,y_train

In [15]:

def run_subject_model(data_path_dict,path,file_name,runs_train,runs_test,mask_list,mask_ind_list,norm_type='NONORM'):
  """
    Function loops over subjects to grap data,scale,data,and run the SVM.
    data_path_dict  : dictionary containing paths to all data stored on AWS
    path : path to save pickle files. NOTE: If run on all subjects, this will take up about 20 GB
    file_name : name of file to prepend to subject id
    runs_train: what runs do we want to train on
    runs_test: what runs do we want to test on
    mask_type: which type of brain mask do we want to apply
    mask_ind: to distinguish between ROI regions of masking(1) and full brain(or full_brain minus ROIs)(0)
    do_psc: do we want to apply Percent Signal Change normalization. Default = True,
    norm: Takes three parameters: none(no normalization will be applied),"SUBJECT"(apply normalization per subject)
          "RUNS"(apply normalization per run separately)
  """
  model_dict = {}
  #get subject information
  subject_ids,subj_paths = get_subj_information(data_path_dict)
  
  #get mask labels to only retrieve time series we care about
  mask_labels_indices,binary_labels,label_type = get_labels(data_path_dict)
  #get mask data
  
  #loop over subjects
  for idx in range(len(subject_ids)):
    sub_id = subject_ids[idx] #get subject id
    subj_path = subj_paths[idx] #get subject path
    #get subject raw data
    #call function to load data and return a dictionary. We are loading .mat files so need to set second param to True
    sub_data_unmasked = access_load_data(mat_path,True)
    #loop over masks indices to reach masks or rois
    for mind in mask_ind_list:   
      mask_data_dict = get_mask_data(data_path_dict,mask_ind_list[mind])
      #loop over masks we want to run
      for mask_type in mask_list:
       
        mask = create_mask(mask_type)
        #mask_data
        sub_data_masked = mask_data(sub_data_unmasked,mask_labels_indices,binary_labels)
        sub_data = sub_data_masked[sub_id]
        #normalize data if indicated and return train data/labels,val data/labels, test data/labels
        normalized_runs = scale_data_single_subj(sub_data,binary_labels,runs_list,norm_type=norm_type)
        #run svm, add clf to dictionary
        clf = run_single_subject_svm(train_data,svc_kernel,svc_c,do_cv=False,params={})
        #run get predicts to get predictions and add to subject_model_dictionary
        final_model_dict = get_predicts(model_dict['model'],val_test_data_labels)
        model_dict[sub_id]['y_test_predicts'] = y_test_predicts
        model_dict[sub_id]['y_test_proba'] = y_test_proba
        model_dict[sub_id]['y_test_dec_func'] = y_test_dec_func
    if len(y_val_predicts)>0:
      model_dict[sub_id]['y_val_predicts'] = y_val_predicts
      model_dict[sub_id]['y_val_proba'] = y_val_proba
      model_dict[sub_id]['y_val_dec_func'] = y_val_dec_func
    destination_path = f'{path}{file_name}{sub_id}.pkl'     
    single_model_path[sub_id] = destination_path
    # filehandler = open(destination_path,"wb") #comment out this
    # pickle.dump(model_dict,filehandler) #comment out this
    # filehandler.close() #comment out this
    object_name = destination_path
    s3_upload(model_dict,object_name,"pickle")
  object_name = f'{path}{mask_type}_data_path_dict.pkl'
  s3_upload(single_model_path,object_name,"pickle")
  # filehandler = open(object_name,"wb") #comment out this
  # pickle.dump(single_model_path,filehandler) #comment out this
  # filehandler.close() #comment out this
  return model_dict

In [18]:

###running model with best params across all masks
##what runs do you want to normalize on
runs_train=['run_02'] #runs we want to train on
runs_test=['run_03','run_04'] #runs we want to test on
runs_list=[2,3,4]
svc_kernel='rbf' #specify kernel 
svc_c = 1 #specify c parameter
save_data_path = f'/content/drive/My Drive/data/singlesubjectmodels/'
mask_list = ['mask']
mask_indices = [0]
#get subject information
subjs_id, subjs_paths = get_subj_information(data_path_dict)
#get mask labels to only retrieve time series we care about
mask_labels_indices,binary_labels,label_type = get_labels(data_path_dict)
subj_mask_model = {}
for idx in range(len(subjs_id)):
  subj_id = subjs_id[idx]
  subj_path = subjs_paths[idx]
  subj_data = access_load_data(subj_path,True)
  for midx in mask_indices:
    mask_dict = get_mask_data(data_path_dict,midx)
    for mask_type in mask_list:
      subj_mask_model[mask_type] = {}
      subj_mask_model[mask_type][subj_id] = {}
      mask = make_mask(mask_dict[mask_type])
      masked_data = mask_subject_data(subj_data,mask,mask_labels_indices)
      scaled_data = scale_data_single_subj(masked_data,runs_train,runs_test,norm='zscore')
      clf,X_train,y_train = run_single_subject_svm(scaled_data,runs_train,binary_labels,svc_kernel,svc_c)
      subj_mask_model[mask_type][subj_id]['model'] = clf
      subj_mask_model[mask_type][subj_id]['X_train'] = X_train
      subj_mask_model[mask_type][subj_id]['y_train'] = y_train
      subj_mask_model[mask_type][subj_id]['predicts'] = get_predicts(clf,scaled_data,runs_test)
      full_path_name = f'{save_data_path}{mask_type}_subject_models.pkl'
      filehandler = open(full_path_name,"wb")
      pickle.dump(subj_mask_model,filehandler)
      filehandler.close()
  


100%|██████████| 4/4 [00:00<00:00,  4.76it/s]


AttributeError: ignored

In [ ]:

###data exploration for different normalization strategies
##what runs do you want to normalize on
runs_train=['run_02'] #runs we want to train on
runs_test=['run_03','run_04'] #runs we want to test on
runs_list=[2,3,4]
##for fMRI, we always want to detrend the data
norm_list = ['psc','zscore','none'] #list of normalization strategies you want to test
save_subject_data =  ['10047_09030','30017_09567'] #specify subjects you want to save for normalization visualizations
svc_kernel='rbf' #specify kernel 
svc_c = 1 #specify c parameter
mask_dict = get_mask_data(data_path_dict,0) #get mask_dict
mask = make_mask(mask_dict['mask']) #create mask
#get subject information
subjs_id, subjs_paths = get_subj_information(data_path_dict)
#get mask labels to only retrieve time series we care about
mask_labels_indices,binary_labels,label_type = get_labels(data_path_dict)
###initialize variables to hold results
model_dict = {}
results = []
df_columns = ['subject_id','norm_type']
#loop over norm_type to get initialize model_dict norm_type dictionary
for norm_type in norm_list:
  model_dict[norm_type] = {}
#for loop to loop over subjects
for idx in range(len(subjs_id)):
  subj_id = subjs_id[idx] #get subj_id
  subj_path = subjs_paths[idx] #get subj_path
  subj_data = access_load_data(subj_path,True) #get subj_data
  masked_data = mask_subject_data(subj_data,mask,mask_labels_indices) #mask subject data
  #loop over normalizations to do
  for norm_type in norm_list:
    norm_results = [subj_id,norm_type] #store subject id and normalization strategy in results list
    scaled_data = scale_data_single_subj(masked_data,runs_train,runs_test,norm=norm_type) #scale the data for runs listed in runs_lit
    clf,X_train,y_train = run_single_subject_svm(scaled_data,runs_train,binary_labels,svc_kernel,svc_c) #run the model
    if subj_id in save_subject_data:    #if subject in list of subjects to save
      model_dict[norm_type][subj_id] = {} #initialize empty dictionary for the subject data
      #store subject variables
      model_dict[norm_type][subj_id]['model'] = clf
      model_dict[norm_type][subj_id]['X_train'] = X_train
      model_dict[norm_type][subj_id]['y_train'] = y_train
      model_dict[norm_type][subj_id]['data'] = scaled_data
    sub_scores,cols = get_accuracy_scores(clf,scaled_data,X_train,y_train,runs_test,binary_labels) #get accuracy scores for analysis
    norm_results.extend(sub_scores) #append to list
    results.append(norm_results) #append to results
df_columns.extend(cols) #extend data frame columns
results_df = pd.DataFrame(results,columns=df_columns) #create df
#save results for analysis
save_data_path = f'/content/drive/My Drive/data/dataexploration/{svc_kernel}_exploration/accuracy_results.csv'
results_df.to_csv(save_data_path)
#save models for visualization
save_models_path = f'/content/drive/My Drive/data/dataexploration/{svc_kernel}_exploration/norm_models.pkl'
filehandler = open(save_models_path,"wb")
pickle.dump(model_dict,filehandler)
filehandler.close()



In [ ]:
def run_subject_model_cv(data_path_dict, destination_path, runs_train, runs_list, mask_type, mask_ind, norm_type='none'):
  """
    Function to run cross validation for data. Note: takes 2 hours to run on all subjects.
    Params:
      data_path_dict  : dictionary containing paths to all data stored on AWS
      path : path to save pickle files. NOTE: If run on all subjects, this will take up about 20 GB
      file_name : name of file to prepend to subject id
      runs_train: what runs do we want to train on
      runs_test: what runs do we want to test on
      mask_type: which type of brain mask do we want to apply
      mask_ind: to distinguish between ROI regions of masking(1) and full brain(or full_brain minus ROIs)(0)
      do_psc: do we want to apply Percent Signal Change normalization. Default = True,
      norm: Takes three parameters: none(no normalization will be applied),"SUBJECT"(apply normalization per subject)
            "RUNS"(apply normalization per run separately)
  Returns: cv_dict, a dictionary that holds cv results
  """

  cv_dict = {}
  subject_ids,subj_paths = get_subj_information(data_path_dict)
  mask_labels_indices,binary_labels,label_type = get_labels(data_path_dict)
  mask_data = get_mask_data(data_path_dict,mask_ind)
  mask = make_mask(mask_data[mask_type])
  for idx in range(len(subject_ids)):
    subj_id = subjs_id[idx]
    subj_path = subjs_paths[idx]
    subj_data = access_load_data(subj_path,True) 
    masked_data = mask_data(subj_data,mask,mask_labels_indices)
     
    sub_data = user_data_dict[sub_id]
    sub_labels = user_data_dict[f"{sub_id}_rt_labels"]
    clf = run_single_subject_svm(sub_data,sub_labels,runs_train,runs_test,norm_type,do_cv=True)
    cv_dict[sub_id] = {}
    cv_dict[sub_id]['model'] = clf
  filehandler = open(destination_path,"wb")
  pickle.dump(cv_dict,filehandler)
  filehandler.close() 
  return cv_dict

In [ ]:
cv_dict = {} #initialize the results dictionary
destination_path = '/content/drive/My Drive/data/dataexploration/cross_validation_results/cv_results.pkl' #where to save data
runs_train=['run_02'] #runs we want to train on
runs_list=[2] #cv results
cv_params = {'C':[1, 5, 10,1000],'kernel':['linear', 'rbf'],'gamma': [.0001, .01, 'auto','scale']}
subject_ids,subj_paths = get_subj_information(data_path_dict)
mask_labels_indices,binary_labels,label_type = get_labels(data_path_dict)
mask_dict = get_mask_data(data_path_dict,0)
mask = make_mask(mask_dict['mask'])
for idx in range(len(subject_ids)):
  subj_id = subjs_id[idx]
  subj_path = subjs_paths[idx]
  subj_data = access_load_data(subj_path,True) 
  masked_data = mask_subject_data(subj_data,mask,mask_labels_indices)
  scaled_data = scale_data_single_subj(masked_data,runs_train,runs_test,norm='zscore')
  clf = run_single_subject_svm(scaled_data,runs_train,binary_labels,do_cv=True,params=cv_params)
  cv_dict[subj_id] = {}
  cv_dict[subj_id]['model'] = clf
filehandler = open(destination_path,"wb")
pickle.dump(cv_dict,filehandler)
filehandler.close() 

100%|██████████| 4/4 [00:00<00:00,  4.40it/s]


In [ ]:
mask_labels_indices,binary_labels,label_type = get_labels(data_path_dict)
mask_dict = get_mask_data(data_path_dict,0)
mask = make_mask(mask_dict['mask'])
mask_labels_indices

array([  3,   4,   5,   6,   7,   8,   9,  15,  16,  17,  18,  19,  20,
        21,  27,  28,  29,  30,  31,  32,  33,  39,  40,  41,  42,  43,
        44,  45,  51,  52,  53,  54,  55,  56,  57,  63,  64,  65,  66,
        67,  68,  69,  75,  76,  77,  78,  79,  80,  81,  87,  88,  89,
        90,  91,  92,  93,  99, 100, 101, 102, 103, 104, 105, 111, 112,
       113, 114, 115, 116, 117, 123, 124, 125, 126, 127, 128, 129, 135,
       136, 137, 138, 139, 140, 141])

In [ ]:
##what runs do you want to normalize on
destination_path = '/content/drive/My Drive/data/dataexploration/cross_validation_results/cv_results.pkl'
runs_train=['run_02'] #runs we want to train on
runs_test=['run_03','run_04'] #runs we want to test on
runs_list=[2]
mask_type = 'mask'
mask_ind = 0
cv_results = run_subject_model_cv(data_path_dict,destination_path,runs_train,runs_list,mask_type,mask_ind,norm='zscore')